<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [103]:
from pymongo import MongoClient
connection = MongoClient(
    host="research.cassee.dev:27017",
    username="read-shark",
    password="msr2021shark")

print(f"Databases present are {connection.list_database_names()}")

db = connection.smartshark_2_1

project = db.project.find_one({})

repos = list(db.vcs_system.find({'project_id':project['_id']}))

print(f"Found {len(repos)} repos")

commits = db.commit.count_documents({'vcs_system_id': {'$in': [x["_id"] for x in repos]}})

print(f"There are {commits} commits for {project['name']}")

Databases present are ['smartshark_2_1']
Found 1 repos
There are 7222 commits for commons-math


In [222]:
import pandas as pd
from bson.json_util import dumps
from json import loads

In [209]:
query = db.pull_request_review.aggregate([{"$lookup": {
    "from": 'pull_request_review',
    "localField": "pull_request_id",
    "foreignField": "_id",
    "as": "pull_request"
}}])
df = pd.DataFrame(list(query))
df.where(df['pull_request'].size > 0).head(10)

ValueError: Array conditional must be same shape as self

In [246]:
db = connection['smartshark_2_1']
print(db)

Database(MongoClient(host=['research.cassee.dev:27017'], document_class=dict, tz_aware=False, connect=True), 'smartshark_2_1')


In [256]:
df_people = pd.DataFrame(list(db.people.find({})))
df_project = pd.DataFrame(list(db.project.find({})))
df_pull_request = pd.DataFrame(list(db.pull_request.find({})))
df_pull_request_review = pd.DataFrame(list(db.pull_request_review.find({})))

In [257]:
df_approved_prs = df_pull_request_review.where(df_pull_request_review['state'] == 'APPROVED').dropna()
df_approved_prs['pull_request_id'] = (df_approved_prs.apply(lambda x: loads(dumps(x['pull_request_id']))['$oid'], axis=1))
df_approved_prs['creator_id'] = (df_approved_prs.apply(lambda x: loads(dumps(x['creator_id']))['$oid'], axis=1))
df_approved_prs.size

170830

In [249]:
df_approved_prs['binary_pr_id'][37]

'5ff1919ccbcaee8d21c67e75'

In [258]:
df_pull_request['pull_request_id'] = (df_pull_request.apply(lambda x: loads(dumps(x['_id']))['$oid'], axis=1))
df_pull_request['creator_id'] = (df_pull_request.apply(lambda x: loads(dumps(x['creator_id']))['$oid'], axis=1))
df_pull_request[['_id', 'pull_request_id']]['pull_request_id'][0]

'5ff1886f3fba52c925be8927'

In [254]:
df_approved_prs__annotated_with_pr_creator = df_approved_prs.join(df_pull_request, lsuffix='_prr', rsuffix='_pr')
df_approved_prs__annotated_with_pr_creator.size

666237

In [259]:
df_approved_prs__annotated_with_pr_creator = pd.merge(df_approved_prs, df_pull_request, on='pull_request_id')

In [260]:
df_approved_prs__annotated_with_pr_creator.head(3)

,_id_x,pull_request_id,external_id_x,state_x,description_x,creator_id_x,submitted_at,author_association_x,commit_sha,pull_request_commit_id,...,source_branch,source_commit_sha,target_repo_url,target_branch,target_commit_sha,source_commit_id,target_commit_id,merge_commit_id,assignee_id,lock_reason
0,5ff1919ecbcaee8d21c67e79,5ff1919ccbcaee8d21c67e75,489446018,APPROVED,,5b4db0dc30a71b06bce03054,2020-09-16 09:32:52,CONTRIBUTOR,a309b67d74c2083d7808d1f7a6aaff22a2a75dd7,5ff1919dcbcaee8d21c67e76,...,WSS-678,a309b67d74c2083d7808d1f7a6aaff22a2a75dd7,https://github.com/apache/ws-wss4j,master,1765c4149d700cac8aea5c1a3b78f2bdee8b6e71,NaN,NaN,NaN,NaN,NaN
1,5ff191a0cbcaee8d21c67e82,5ff1919ecbcaee8d21c67e7f,479608653,APPROVED,,5b4db0dc30a71b06bce03054,2020-09-01 10:28:43,CONTRIBUTOR,c21157ad10224d78291b3433914847f9227f038d,5ff1919fcbcaee8d21c67e80,...,wss4j-ws-security-common,c21157ad10224d78291b3433914847f9227f038d,https://github.com/apache/ws-wss4j,master,e9c20e63dc1694dbc5e116e4c9e82190aa81320a,NaN,NaN,NaN,NaN,NaN
2,5ff191a3cbcaee8d21c67e97,5ff191a2cbcaee8d21c67e93,396451248,APPROVED,,5b4db0dc30a71b06bce03054,2020-04-20 13:27:33,CONTRIBUTOR,fd69b86d8ebdf589fc3c7eabd9e738a019d30653,5ff191a3cbcaee8d21c67e94,...,master,fd69b86d8ebdf589fc3c7eabd9e738a019d30653,https://github.com/apache/ws-wss4j,master,507c7a00dd31cca286d0d3d7451cb7da068f926a,NaN,NaN,NaN,NaN,NaN


In [265]:
# Get the PR's that were merged by someone that is not the same person as the one that opened the request
df_approved_prs__annotated_with_pr_creator.where(df_approved_prs__annotated_with_pr_creator['creator_id_x'] != df_approved_prs__annotated_with_pr_creator['creator_id_y'])[['creator_id_x', 'creator_id_y']]

,creator_id_x,creator_id_y
0,5b4db0dc30a71b06bce03054,5f0d6a7baf02e2d6de888cd0
1,5b4db0dc30a71b06bce03054,5f0d6a7baf02e2d6de888cd0
2,5b4db0dc30a71b06bce03054,5ff191a2c26a57681e7b9533
3,5b4db0dc30a71b06bce03054,5ff191a2c26a57681e7b9533
4,5b4db0dc30a71b06bce03054,5f0d6a7baf02e2d6de888cd0
...,...,...
17078,5b4db0dc30a71b06bce03054,5f0d6a7baf02e2d6de888cd0
17079,5b4db0dc30a71b06bce03054,5f0d6a7baf02e2d6de888cd0
17080,59bc2341f2a4565fe9cfaace,60fd84a0f73e2aa3902dad2a
17081,59bc2341f2a4565fe9cfaace,60fd84a5f73e2aa3902dc6d2


bson.objectid.ObjectId